# Intro

It's time for me to start learning about agents.


## What are agents?

![[image source: Tweet from Abhishek Thakur](https://x.com/abhi1thakur/status/1873697074405122144)](imgs/say_agentic_one_more_time.jpeg)

Let's start with some definitions of agents from different sources.


::: {.callout-note}
## Agent Definition from LangChain Blog Post - [source](https://blog.langchain.dev/what-is-an-agent/)
An AI agent is a system that uses an LLM to decide the control flow of an application.
:::

::: {.callout-note}
## Agent Definition from AWS - [source](https://aws.amazon.com/what-is/ai-agents/)
An artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals.
:::

::: {.callout-note}
## Agent Definition from Chip Huyen's Book "AI Engineering" - [source](https://learning.oreilly.com/library/view/ai-engineering/9781098166298/ch06.html#ch06_agents_1730157386572111)

An **agent** is anything that can perceive its environment and act upon that environment. This means that an **agent** is characterized by the environment it operates in and the set of actions it can perform.
:::



::: {.callout-note}
## Agent Definition from Mongo DB Blog Post - [source](https://www.mongodb.com/resources/basics/artificial-intelligence/ai-agents#what-is-an-ai-agent)

An AI **agent** is a computational entity with an awareness of its environment that’s equipped with faculties that enable perception through input, action through tool use, and cognitive abilities through foundation models backed by long-term and short-term memory.
:::

::: {.callout-note}
## Agent Definition from Anthropic - [source](https://www.anthropic.com/research/building-effective-agents)

**"Agent"** can be defined in several ways. Some customers define agents as fully autonomous systems that operate independently over extended periods, using various tools to accomplish complex tasks. Others use the term to describe more prescriptive implementations that follow predefined workflows. At Anthropic, we categorize all these variations as **agentic** systems, but draw an important architectural distinction between **workflows** and **agents**:

**Workflows** are systems where LLMs and tools are orchestrated through predefined code paths.

**Agents**, on the other hand, are systems where LLMs dynamically direct their own processes and tool usage, maintaining control over how they accomplish tasks.
:::


::: {.callout-note}
## Agent Definition from Hugging Face Blog Post on `smolagents` - [source](https://huggingface.co/blog/smolagents)

Any efficient system using AI will need to provide LLMs some kind of access to the real world: for instance the possibility to call a search tool to get external information, or to act on certain programs in order to solve a task. In other words, LLMs should have agency. Agentic programs are the gateway to the outside world for LLMs.


**Agents** are programs where LLM outputs control the workflow. Note that with this definition, **"agent"** is not a discrete, 0 or 1 definition: instead, "agency" evolves on a continuous spectrum, as you give more or less power to the LLM on your workflow.
:::


## Is it an Agent? Is it Agentic? It's more like a spectrum with a lot of gray area!

![[image source: Tweet from Andrew Ng](https://x.com/AndrewYNg/status/1801295202788983136)](imgs/its_agentic.jpg)

There is a lot of debate and discussion on what exactly is an agent and what is not an agent. I think there is a lot of gray area here and something we 
have to just accept, at least for now. I think Andrew Ng makes some really good points in this [tweet](https://x.com/AndrewYNg/status/1801295202788983136). As Andrew points out, rather than engaging in binary debates about whether something qualifies as a "true agent," we should think about systems as existing on a spectrum of agent-like qualities.  The adjective "agentic" itself becomes particularly useful here, allowing us to describe systems that incorporate agent-like patterns to different degrees without getting caught in restrictive definitions.

This spectrum-based view is reinforced by Anthropic's recent [blog post](https://www.anthropic.com/research/building-effective-agents) on agents. They acknowledge that while they draw an architectural distinction between workflows (systems with predefined code paths) and agents (systems with dynamic control), they categorize both under the broader umbrella of "agentic systems." Similarly, we saw from one of our definitions above that "agent" isn't a discrete, 0 or 1 definition, but rather evolves on a continuous spectrum as you give more or less power to the LLM in your system. This aligns with Andrew Ng's observation that there's a gray zone between what clearly is not an agent (prompting a model once) and what clearly is (an autonomous system that plans, uses tools, and executes multiple steps independently).

![[image source: Blog post from Nathan Lambert on the AI Agent Spectrum](https://www.interconnects.ai/p/the-ai-agent-spectrum)](imgs/llm_loop_is_agent.webp)

Nathan Lambert also writes about the AI agent spectrum in this [blog post](https://www.interconnects.ai/p/the-ai-agent-spectrum). Nathan discusses that the simplest system on this spectrum would be any tool-use language model and that the spectrum of agents increases in complexity from there. I like how Nathan makes the point that the spectrum will continue to evolve and that the definition of an agent will continue to change as the field evolves. Over time, certain technologies will reach milestones where they become definitive examples of AI agents. Therefore at some point, basic tool use with an LLM may not be considered an agent, even though it's the basic starting point on the agentic spectrum.

![[image source: Tweet from Hamel Husain](https://x.com/HamelHusain/status/1867289397898752222)](imgs/llm_workflow_vs_loop.png)

Personally, agents and agentic workflows are still so new to me and I have a lot to learn on this topic. I have deployed LLMs in production as well as built some applications where LLMs use function calling (tools) within a conversational chat interface. So I think some of my previous work has fallen somewhere within this AI agentic spectrum, even if it's at one end of the spectrum. I'm going to  keep an open mind and avoid getting caught up in debates about categorical definitions. I'll try to avoid the hype and marketing fluff but be on the lookout for innovation and practical applications. 



# The Tool Calling Loop: The Foundation of Agentic Systems


![[image source: Tweet from Abhishek Thakur](https://x.com/abhi1thakur/status/1875159964785987904)](imgs/just_loops_meme.jpeg)

So where do we even start on this spectrum of AI agents? Practically, I think the first step is to start with an LLM equipped with tools.
I think this is what Anthropic refers to as the ["The augmented LLM"](https://www.anthropic.com/research/building-effective-agents).

![[image source: Blog post from Anthropic on Building effective agents](https://www.anthropic.com/research/building-effective-agents)](imgs/the_augmented_llm.png)

This is the building block, an LLM equipped with tools. I think we need to take it slightly further and make it clear we need a tool calling loop. There are two distinct loops here
I want to discuss. The entire process is kicked off by sending the user request to the LLM. The LLM then decides on the initial tool calls. These tool calls can be executed in parallel since the results of each tool call are independent of one another. The second loop is to handle follow up tool calls. These are tool calls that are dependent on the results of previous tool calls. Implementing this logic together is what I refer to as the "tool calling loop". 

I wrote about this tool calling loop in a previous [blog_post](https://drchrislevy.github.io/posts/anthropic/anthropic.html#my-own-wrapper-with-an-emphasis-on-the-tool-calling-loop). Here is an image I created during the writing of that post to illustrate the concept. 

![[image source: previous blog post](https://drchrislevy.github.io/posts/anthropic/anthropic.html#my-own-wrapper-with-an-emphasis-on-the-tool-calling-loop)](imgs/llm_tool_call_loop.png)


Here is one such implementation of the tool calling loop. It assumes the typical JSON format for the tool calls and uses the OpenAI chat completion API format.
I'm using the [`litellm`](https://github.com/BerriAI/litellm) library to call the OpenAI API since I can easily switch to another model (such as Anthropic)
and still use the same OpenAI API format. If you have never used `litellm` before that is fine! This is my first time using it.
All you need to know is that `from litellm import completion` is the same as calling `chat.completions.create(...)` from the `openai` library.


In [1]:
# | echo: false
# | warning: false
import dotenv

dotenv.load_dotenv()


from IPython.display import Markdown, display


def import_python_as_markdown(file_path):
    with open(file_path, "r") as file:
        content = file.read()
    return f"```python\n{content}\n```"


from tool_calling_loop import llm_with_tools, run_step

/Users/christopher/personal_projects/DrChrisLevy.github.io/posts/agents/env/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [2]:
# | echo: false
file_path = "tool_calling_loop.py"
markdown_content = import_python_as_markdown(file_path)
display(Markdown(markdown_content))

```python
import json
from concurrent import futures
from typing import Any, Callable, Dict

from litellm import completion
from utils import (
    console_print_llm_output,
    console_print_step,
    console_print_tool_call_inputs,
    console_print_tool_call_outputs,
    console_print_user_request,
)


def call_tool(tool: Callable, tool_args: Dict) -> Any:
    return tool(**tool_args)


def run_step(messages, tools=None, tools_lookup=None, model="gpt-4o-mini", **kwargs):
    response = completion(model=model, messages=messages, tools=tools, **kwargs)
    response_message = response.choices[0].message.model_dump()
    response_message.pop("function_call", None)
    tool_calls = response_message.get("tool_calls", [])
    messages.append(response_message)

    if not tool_calls:
        response_message.pop("tool_calls", None)
        return messages

    tools_args_list = [json.loads(t["function"]["arguments"]) for t in tool_calls]
    tools_callables = [tools_lookup[t["function"]["name"]] for t in tool_calls]
    tasks = [(tools_callables[i], tools_args_list[i]) for i in range(len(tool_calls))]
    console_print_tool_call_inputs(tool_calls)
    with futures.ThreadPoolExecutor(max_workers=10) as executor:
        tool_results = list(executor.map(lambda p: call_tool(p[0], p[1]), tasks))
    console_print_tool_call_outputs(tool_calls, tool_results)
    for tool_call, tool_result in zip(tool_calls, tool_results):
        messages.append(
            {
                "tool_call_id": tool_call["id"],
                "role": "tool",
                "content": str(tool_result),
                "name": tool_call["function"]["name"],
            }
        )
    return messages


def llm_with_tools(messages, tools=None, tools_lookup=None, model="gpt-4o-mini", max_steps=10, **kwargs):
    console_print_user_request(messages, model)
    done_calling_tools = False
    for counter in range(max_steps):
        console_print_step(counter)
        messages = run_step(messages, tools, tools_lookup, model=model, **kwargs)
        done_calling_tools = messages[-1]["role"] == "assistant" and messages[-1].get("content") and not messages[-1].get("tool_calls")
        if done_calling_tools:
            break
    console_print_llm_output(messages[-1]["content"])
    return messages

```

Let's go through some examples to see how this works.
First we will run a single step which is a single LLM call.
We will not add any tools to this LLM call.

In [3]:
messages = [
    {"role": "user", "content": "Hello friend!"},
]
run_step(messages)

[{'role': 'user', 'content': 'Hello friend!'},
 {'content': 'Hello! How can I assist you today?', 'role': 'assistant'}]

Now let's define some tools.
These tools are a list of functions that we can call.
We also have a lookup dictionary that maps the tool name to the tool function.

In [4]:
from tools import TOOL_LKP, TOOLS

TOOL_LKP

{'web_search': <function tools.web_search(query: str) -> str>,
 'execute_python_code': <function python_sandbox.execute_python_code(code: str, sandbox=None) -> dict>,
 'visit_web_page': <function tools.visit_web_page(url)>}

Let's see how each tool works first.
This tool allows me to execute arbitrary python code. It's actually running in a [Modal Sandbox](https://modal.com/docs/guide/sandbox). 
It's an awesome feature of Modal where you can run arbitrary code in a secure environment. For now, just think of it as a way to execute python code
and get back the results.

In [5]:
TOOL_LKP["execute_python_code"](code="print('Hello World!')")

{'stdout': 'Hello World!\n',
 'stderr': '',
 'success': True,
 'result': 'None',
 'error': None}

In [6]:
TOOL_LKP["execute_python_code"](code="import math; x = math.sqrt(4); print(x); y=2; x-y")

{'stdout': '2.0\nOut[1]: 0.0\n',
 'stderr': '',
 'success': True,
 'result': '0.0',
 'error': None}

The next tool uses `duckduckgo-search` to search the web.

In [7]:
TOOL_LKP["web_search"](query="What sporting events are happening today?")

[{'title': 'Live Sports On TV Today - TV Guide',
  'href': 'https://www.tvguide.com/sports/live-today/',
  'body': "Live Sports on TV Today. Here's sports to watch today, Friday, Jan 3, 2025. ... A mixed team tennis event features players from 18 countries competing in singles and doubles as they vie for the ..."},
 {'title': 'Sports on TV Today - Sports Media Watch',
  'href': 'https://www.sportsmediawatch.com/sports-on-tv-today-games-time-channel/',
  'body': 'See where to watch sports on TV today with this daily, updated guide of games and events on TV and streaming. This site may earn commission on subscriptions purchased via this page. For a full list of sports TV schedules, see this page. Games on TV Today (Friday, January 3) All times Eastern (ET)'},
 {'title': "Today's Top Sports Scores and Games (All Sports) | FOX Sports",
  'href': 'https://www.foxsports.com/scores',
  'body': "Visit FOXSports.com for today's top sports scores and games. Explore real-time game scores across M

And the next tool visits a web page and converts it to markdown.

In [8]:
print(TOOL_LKP["visit_web_page"](url="https://drchrislevy.github.io/"))

Chris Levy

[Chris Levy](./index.html)

* [About](./index.html)
* [Blog](./blog.html)

 
 

## On this page

* [About Me](#about-me)

# Chris Levy

 
[twitter](https://twitter.com/cleavey1985)
[Github](https://github.com/DrChrisLevy)
[linkedIn](https://www.linkedin.com/in/chris-levy-255210a4/)

**Hello!** I’m Chris Levy. I work in ML/AI and backend Python development.

## About Me

I spent a good amount of time in school where I completed a PhD in applied math back in 2015. After graduating I shifted away from academia and started working in industry. I mostly do backend python development these days, and build ML/AI applications/services. I work across the entire stack from research, to training and evaluating models, to deploying models, and getting in the weeds of the infrastructure and devops pipelines.

Outside of AI/ML stuff, I enjoy spending time with my family and three kids, working out, swimming, cycling, and playing guitar.

![](pic_me.jpeg)


To pass these tools to the LLM, we use the typical JSON format used within the OpenAI API format.

In [9]:
TOOLS

[{'type': 'function',
  'function': {'name': 'execute_python_code',
   'description': 'Run and execute the python code and return the results.',
   'parameters': {'type': 'object',
    'properties': {'code': {'type': 'string',
      'description': 'The python code to execute.'}},
    'required': ['code']}}},
 {'type': 'function',
  'function': {'name': 'web_search',
   'description': 'Search the web for the query and return the results.',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'The query to search for.'}},
    'required': ['query']}}},
 {'type': 'function',
  'function': {'name': 'visit_web_page',
   'description': 'Visit the web page and return the results.',
   'parameters': {'type': 'object',
    'properties': {'url': {'type': 'string',
      'description': 'The URL to visit.'}},
    'required': ['url']}}}]

Okay so let's run the tool calling loop now with the tools defined above to illustrate how it works.

In [11]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant. Use the supplied tools to assist the user. Always use python to do math. Todays date is 2025-01-03.",
    },
    {
        "role": "user",
        "content": """
        Recently Steph Curry made a series of 3 pointers in one game without missing. 
        How many did he make?
        How many points did he score in total?
        Calculate the total points scored by both teams and then the percentage of those points that were made by Steph Curry alone.

        Also, how old is Lebron James and how many points did he score in his game on Jan 2 2025
        """,
    },
]
messages_final = llm_with_tools(messages, tools=TOOLS, tools_lookup=TOOL_LKP)

╭───────────────────────────────────────────────── User Request ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│         Recently Steph Curry made a series of 3 pointers in one game without missing.                           │
│         How many did he make?                                                                                   │
│         How many points did he score in total?                                                                  │
│         Calculate the total points scored by both teams and then the percentage of those points that were made  │
│ by Steph Curry alone.                                                                                           │
│                                                                                                                 │
│         Also, how old is Lebron James and how many points did he score in his game on Jan 2 2025                │
│                                                                                                                 │
│                                                                                                                 │
╰─ gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭────────────────────────────────────────── Parallel Tool Calls Inputs ───────────────────────────────────────────╮
│ ╭───────────────────────────────────────────────── Tool Call ─────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ │ {'query': 'Steph Curry 3 pointers made January 2025'}                                                       │ │
│ │                                                                                                             │ │
│ ╰─ web_search - call_dfAjeOJDTTughbsNgUOYF5wx ────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────────── Tool Call ─────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ │ {'query': 'LeBron James age January 2025'}                                                                  │ │
│ │                                                                                                             │ │
│ ╰─ web_search - call_UDBwAtEdLMHYQsSnWuC8hKkM ────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────────── Tool Call ─────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ │ {'query': 'LeBron James points scored January 2 2025'}                                                      │ │
│ │                                                                                                             │ │
│ ╰─ web_search - call_C5l3TNbHlmm2hjWwUgV9Copf ────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Parallel Tool Calls Outputs ──────────────────────────────────────────╮
│ ╭───────────────────────────────────────────── Tool Call Output ──────────────────────────────────────────────╮ │
│ │ Result 1:                                                                                                   │ │
│ │ Title: NBA: Stephen Curry makes history as Golden State Warriors rout ... - BBC                             │ │
│ │ URL: https://www.bbc.com/sport/basketball/articles/crk034g1levo                                             │ │
│ │ Stephen Curry sets a new three-pointer record as the Golden State Warriors defeat the Philadelphia 76ers    │ │
│ │ 139-105 in the NBA. ... 3 January 2025. ... C...                                                            │ │
│ │                                                                                                             │ │
│ │ Result 2:                                                                                                   │ │
│ │ Title: Steph Curry Makes History As The Golden State Warriors Crush The ...                                 │ │
│ │ URL: https://seattlemedium.com/curry-three-pointers-nba/                                                    │ │
│ │ Steph Curry sets NBA record with 8 three-pointers without a miss in a 30-point, 10-assist game. ... January │ │
│ │ 3, 2025. Curry leads the NBA for most thre...                                                               │ │
│ │                                                                                                             │ │
│ │ Result 3:                                                                                                   │ │
│ │ Title: Steph Curry perfect from 3 as Warriors blow out 76ers - The Mercury News                             │ │
│ │ URL:                                                                                                        │ │
│ │ https://www.mercurynews.com/2025/01/02/curry-perfect-from-3-as-warriors-start-2025-hot-in-blowout-win-over- │ │
│ │ 76ers/                                                                                                      │ │
│ │ Golden State Warriors' Stephen Curry (30) makes a back-to-back three-point basket against the Philadelphia  │ │
│ │ 76ers in the fourth quarter of an NBA game ...                                                              │ │
│ │                                                                                                             │ │
│ │ Result 4:                                                                                                   │ │
│ │ Title: Stephen Curry tastes 3-point perfection as Warriors pound 76ers                                      │ │
│ │ URL:                                                                                                        │ │
│ │ https://www.reuters.com/sports/basketball/stephen-curry-tastes-3-point-perfection-warriors-pound-76ers-2025 │ │
│ │ -01-03/                                                                                                     │ │
│ │ Stephen Curry added to his 3-point-shooting lore Thursday night, making all eight of his attempts for the   │ │
│ │ first time in his career as the Golden State...                                                             │ │
│ │                                                                                                             │ │
│ │ Result 5:                                                                                                   │ │
│ │ Title: Stephen Curry's 3-point perfection caps an anomalous win the Warriors ...                            │ │
│ │ URL: https://www.nytimes.com/athletic/6035422/2025/01/03/stephen-curry-warriors-8-for-8-3-pointers-76ers/   │ │
│ │ Stephen Curry's 3-point perfection caps an anomalous win the Warriors needed. ... January 3, 2025. ... His  │ │
│ │ average of 4.3 made 3-pointers per game is ...      

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭────────────────────────────────────────── Parallel Tool Calls Inputs ───────────────────────────────────────────╮
│ ╭───────────────────────────────────────────────── Tool Call ─────────────────────────────────────────────────╮ │
│ │    1 # Calculating total points scored by Steph Curry based on his made three-pointers                      │ │
│ │    2 three_pointers_made = 8                                                                                │ │
│ │    3 points_per_three_pointer = 3                                                                           │ │
│ │    4 steph_points = three_pointers_made * points_per_three_pointer                                          │ │
│ │    5                                                                                                        │ │
│ │    6 # Total points scored in the game                                                                      │ │
│ │    7 lakers_points = 114                                                                                    │ │
│ │    8 blazers_points = 106                                                                                   │ │
│ │    9 total_points = lakers_points + blazers_points                                                          │ │
│ │   10                                                                                                        │ │
│ │   11 # Calculating percentage of total points scored by Steph Curry                                         │ │
│ │   12 curry_percentage = (steph_points / total_points) * 100                                                 │ │
│ │   13                                                                                                        │ │
│ │   14 steph_points, total_points, curry_percentage                                                           │ │
│ ╰─ execute_python_code - call_cleFLg5HXRIO5F0VYAZXF6S0 ───────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Parallel Tool Calls Outputs ──────────────────────────────────────────╮
│ ╭───────────────────────────────────────────── Tool Call Output ──────────────────────────────────────────────╮ │
│ │ stdout:                                                                                                     │ │
│ │ Out[1]: (24, 220, 10.909090909090908)                                                                       │ │
│ │                                                                                                             │ │
│ │ is_success: True                                                                                            │ │
│ ╰─ execute_python_code - call_cleFLg5HXRIO5F0VYAZXF6S0 ───────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭───────────────────────────────────────────────── Final Answer ──────────────────────────────────────────────────╮
│ Here are the details from Steph Curry's recent game:                                                            │
│                                                                                                                 │
│  • Three-pointers made by Steph Curry: 8                                                                        │
│  • Total points scored by Steph Curry: 24 points (8 three-pointers × 3 points each)                             │
│  • Total points scored by both teams: 220 points (Lakers 114 + Blazers 106)                                     │
│  • Percentage of total points made by Steph Curry: Approximately 10.91%                                         │
│                                                                                                                 │
│ As for LeBron James:                                                                                            │
│                                                                                                                 │
│  • LeBron James age: 40 years (as he turned 40 on January 1, 2025)                                              │
│  • Points scored by LeBron James on January 2, 2025: 38 points                                                  │
│                                                                                                                 │
│ If you have any more questions or need further assistance, feel free to ask!                                    │
╰─ final-answer ──────────────────────────────────────────────────────────────────────────────────────────────────╯

We can look at all the messages in the final output:


In [12]:
from pprint import pprint

pprint(messages_final)

[{'content': 'You are a helpful assistant. Use the supplied tools to assist '
             'the user. Always use python to do math. Todays date is '
             '2025-01-03.',
  'role': 'system'},
 {'content': '\n'
             '        Recently Steph Curry made a series of 3 pointers in one '
             'game without missing. \n'
             '        How many did he make?\n'
             '        How many points did he score in total?\n'
             '        Calculate the total points scored by both teams and then '
             'the percentage of those points that were made by Steph Curry '
             'alone.\n'
             '\n'
             '        Also, how old is Lebron James and how many points did he '
             'score in his game on Jan 2 2025\n'
             '        ',
  'role': 'user'},
 {'content': None,
  'role': 'assistant',
  'tool_calls': [{'function': {'arguments': '{"query": "Steph Curry 3 pointers '
                                            'made Januar

# TODO Thoughts

- the idea of agent writing code instead of just JSON. As discussed in the [Hugging Face Blog Post on `smolagents`](https://huggingface.co/blog/smolagents) as well as the paper [Executable Code Actions Elicit Better LLM Agents](https://huggingface.co/papers/2402.01030)
- HF collection of papers on agents - [source](https://huggingface.co/collections/m-ric/agents-65ba776fbd9e29f771c07d4e)
- [ReAct Paper - main paper on agents?](https://huggingface.co/papers/2210.03629)
- [Secure Code Execution with E2B](https://huggingface.co/docs/smolagents/tutorials/secure_code_execution)
    - But I would like to look into Modal Sand Box too
- [Which AI Agent framework should i use? (CrewAI, Langgraph, Majestic-one and pure code)](https://medium.com/@aydinKerem/which-ai-agent-framework-i-should-use-crewai-langgraph-majestic-one-and-pure-code-e16a6e4d9252)
- [321 real-world gen AI use cases from the world's leading organizations](https://cloud.google.com/transform/101-real-world-generative-ai-use-cases-from-industry-leaders)
    - maybe some interesting ideas here for agents?
- a single LLM call is not agentic!
- an augmented LLM with tools may not be an agent but rather on the agentic spectrum, according to anthropic blog post.
- one cool thing about multi agents is that each agent is modular and can be good at one thing. Also these agents can be different models/prompts/providers etc.
- from CrewAI - role playing, focus, tools, cooperation, guardrails, memory
    - role playing - give the agent a role/persona with system prompt, careful consideration of role and backstory for the agent
    - focus - Too much tools, too much context, etc. Make agent focus on one thing. Don't let agent do too much. 
    - tools - too many tools or overload of tools can lead to issues. Choose the tools carefully for each agent and not too many.
    - cooperation - multi agent cooperation. Just like human can intteract with ChatGPT in iterative manner to get better results/outputs, the LLM agents can interact with each other anc accept feedback, delegate tasks.
    - guardrails - agents getting stuck doing loops, derailing, etc. Many frameworks have guardrails built in.
    - memory - ability for the agent to remember what it has done, what it has learned, etc. Short term memory is mmemory during execution of a task. share knowledge, activities. and learnings with agents. sharing intermedia information even before task completion. Long term memory - after the crew is finished. Stored in DB. LOng term memory can be used in future tasks. Leads to self improving agents.
    - process i.e. process.hierarchical
- CrewAI advice:
    - think like a manager 
    - Cache tool responses across all agents 
    - Tools should be versatile and fault tolerant 
    - I feel like this is simply good SWE practice.


# Frameworks and Platforms

- crewAI
- Langgraph
- Smolagents
- autogen
- [Amazon Bedrock Agents](https://aws.amazon.com/bedrock/agents/)

# Resources

[OpenAI Note Book - Prerequisite to Swarm - Orchestrating Agents: Routines and Handoffs](https://github.com/openai/openai-cookbook/blob/main/examples/Orchestrating_agents.ipynb)

[Anthropic Blog - Building effective agents](https://docs.anthropic.com/en/docs/build/agents/building-effective-agentshttps://www.anthropic.com/research/building-effective-agents)

[AI Engineering Book - By Chip Huyen](https://learning.oreilly.com/library/view/ai-engineering/9781098166298/)

[Hugging Face Blog Post - Introducing smolagents, a simple library to build agents](https://huggingface.co/blog/smolagents)

[litelllm - GIthub](https://github.com/BerriAI/litellm)

[Executable Code Actions Elicit Better LLM Agents](https://huggingface.co/papers/2402.01030)

[Hugging Face Collection of Papers on Agents](https://huggingface.co/collections/m-ric/agents-65ba776fbd9e29f771c07d4e)

[Deep Learning AI Course -  Multi AI Agent Systems with crewAI](https://www.deeplearning.ai/short-courses/multi-ai-agent-systems-with-crewai/)


[Raising the bar on SWE-bench Verified with Claude 3.5 Sonnet](https://www.anthropic.com/research/swe-bench-sonnet)

[Building Effective Agents Cookbook](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents)

[deeplearning.ai - The Batch - Issue 281 - Recap of 2024 - with some agentic thoughts](https://www.deeplearning.ai/the-batch/issue-281/)

[LangChain Blog Post - What is an Agent?](https://blog.langchain.dev/what-is-an-agent/)


[Lindy](https://www.lindy.ai/)


["Say Agentic One More Time" Meme](https://x.com/abhi1thakur/status/1873697074405122144)


[AWS: What is an Agent?](https://aws.amazon.com/what-is/ai-agents/)

[Chapter 6 Agents from AI Engineering Book by Chip Huyen](https://learning.oreilly.com/library/view/ai-engineering/9781098166298/ch06.html#ch06_agents_1730157386572111)

[tweet from Andrew Ng on AI Agent Spectrum](https://x.com/AndrewYNg/status/1801295202788983136)

[Nathan Lambert Blog Post on the AI Agent Spectrum](https://www.interconnects.ai/p/the-ai-agent-spectrum)

[Lang Chain Academy - AI Agents with LangGraph](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

[Deep Learning AI Course - LangGraph AI Agents](https://www.deeplearning.ai/short-courses/ai-agents-in-langgraph/)


[A simple Python implementation of the ReAct pattern for LLMs](https://til.simonwillison.net/llms/python-react-pattern)

[AI Agents That Matter](https://agents.cs.princeton.edu/)

[Demystifying AI Agents: A Guide for Beginners](https://www.mongodb.com/resources/basics/artificial-intelligence/ai-agents)


